In [1]:
!pip install pdfplumber


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pdfplumber
import os
import csv
import logging

In [3]:
from dotenv import load_dotenv

load_dotenv()

input_path = os.getenv("INPUT_PATH")
output_path = os.getenv("OUTPUT_PATH")
password = os.getenv("PASSWORD")

In [4]:
input_path

'./archive/sample.pdf'

In [5]:
def extract_tables_from_pdf(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # # Extract text from the first page
            # first_page = pdf.pages[0]
            # text = first_page.extract_text()

            # # Assuming the heading is the first line of text in the PDF
            # heading = text.split('\n')[0]
            # cleaned_heading = re.sub(r'[\\/*?:"<>|]', "", heading)

            # Define the output CSV file path
            pdf_pathghh = pdf_path.split('/')[-1].split('_')[1]
            csv_filename = f"{pdf_pathghh}.csv"
            tables = []
            for page in pdf.pages:
                table = page.extract_table()
                if table:
                    tables.append(table)
            return tables, csv_filename
    except Exception as e:
        logging.error(f"Error occured extracting from pdf {pdf_path}",{e})

In [6]:
def save_table_to_csv(tables, csv_filename, output_dir):
    try:
        csv_path = os.path.join(output_dir, csv_filename)
        with open(csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            for table in tables:
                for row in table:
                    writer.writerow(row)
        logging.info(f"Saved CSV: {csv_path}")
    except Exception as e:
        logging.error(f"Error occured saving table to csv {csv_filename}",{e})

In [7]:
def process_pdfs_in_directory(directory_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    # List all PDF files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_file_path = os.path.join(directory_path, filename)
            logging.info(f"Processing: {pdf_file_path}")
            tables, csv_filename = extract_tables_from_pdf(pdf_file_path)
            if tables:
                save_table_to_csv(tables, f"{csv_filename}", output_dir)
            else:
                logging.info(f"No tables found in: {pdf_file_path}")

In [8]:
from PyPDF2 import PdfReader, PdfWriter

# Decrypt the PDF
def decrypt_pdf(input_path, output_path, password):
    reader = PdfReader(input_path)
    if reader.is_encrypted:
        reader.decrypt(password)
    
    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)
    
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

In [9]:
decrypt_pdf(input_path, output_path, password)

In [11]:
# pdf_directory_path = './data/decrypted/2023_sbi/'
# output_directory_path = './data/output/2023_sbi/'
pdf_directory_path = './archive/decrypted/'
output_directory_path = './archive/output/'
process_pdfs_in_directory(pdf_directory_path, output_directory_path)

In [13]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("./data/output/2023_sbi/08.pdf.csv")
df.head()

,Txn Date,Value\nDate,Description,Ref No./Cheque\nNo.,Debit,Credit,Balance
0,3 Aug 2023,3 Aug 2023,BULK POSTING-ACHCr\nNACH00000000001792 REG23\n-,NaN,NaN,125.40,"6,72,754.56"
1,3 Aug 2023,3 Aug 2023,BULK POSTING-ACHCr\nHDFC01075000010329 TCIL-,NaN,NaN,9.00,"6,72,763.56"
2,4 Aug 2023,4 Aug 2023,BULK POSTING-ACHCr\nUTIB00788000017240 KFIL\nF...,NaN,NaN,16.00,"6,72,779.56"
3,5 Aug 2023,5 Aug 2023,BY TRANSFER-INB\nIMPS321727038666/85214796\n32...,MAD00035329732\n2\nMAD00035329732\n2,NaN,"1,15,760.21","7,88,539.77"
4,6 Aug 2023,6 Aug 2023,BULK POSTING-ACHCr\nNACH00000000016326\nNARAYA...,NaN,NaN,7.00,"7,88,546.77"


In [15]:
len(df)
unique_transactions = df["Description"].unique()
len(unique_transactions)

38